**YOUR NAMES HERE**

CS343: Neural Networks

Project 3: Convolutional Neural Networks

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt

plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=7)

# Automatically reload your external source code
%load_ext autoreload
%autoreload 2

## Goal

The goal of this notebook is to walk you through the recommended implementation order for your convolutional neural network and provide test code to help make sure your implementation works as expected every step of the way.

Next week, you will test your network on the STL-10 dataset.

**Note:** You will be using this codebase again for one later project so it is worth making sure that you understand how the classes and methods are organized and that you write efficient, clear code.

**Global note: Make sure any debug printouts do not appear if `verbose=False`!**


## Task 4: Building a convolutional neural network

Now that you have the core convolution and max pool operations implemented, you can tackle the main task of building a convolutional neural network. This will be a "deep" 4-layer neural network with the following architecture:

1. Convolution (net-in), Relu (net-act).
2. Max pool 2D (net-in), linear (net-act).
3. Dense (net-in), Relu (net-act).
4. Dense (net-in), soft-max (net-act).

In the above outline, the first part is the layer net-in type (e.g. conv, maxpool, etc) and the second is the layer's activation function (rectified linear (Relu), soft-max, etc). 

Unlike the MLP project, your network will adopt an object-oriented, modular design that should make it straightforward to add/remove/customize layers with minimal code changes.

### 4a. Migrate existing code (`layer.py`)

Copy-paste your code from the last project to implement the following functions:

- `one_hot`

### 4b. Network layer activation functions (`layer.py`)

Implement the following activation functions. Remember, an activation function transforms a layer's "net input" to "net activation".

- `linear`
- `relu`
- `softmax`
- `compute_net_act`

Equation for softmax:

$e^{x_{ij}} / \sum_{k=1}^C e^{x_{ik}}$ where $x_{ij}$ is the "net in" value for neuron $j$ for input $i$. $C$ corresponds to the number of classes in the dataset.

In [ ]:
from layer import *

#### Test: `linear()`

In [ ]:
test_layer = Layer(0, 'test')
test_layer.net_in = np.arange(10)
test_layer.linear()
print(f'{test_layer.net_act} should be [0 1 2 3 4 5 6 7 8 9]')

#### Test: `relu()`

In [ ]:
np.random.seed(0)
test_layer = Layer(0, 'test')
test_layer.net_in = np.random.rand(3, 3) - 0.5
test_layer.relu()
print(f'{test_layer.net_act}')

You should get:

`[[0.0488135 0.2151894 0.1027634]
 [0.0448832 0.        0.1458941]
 [0.        0.391773  0.4636628]]`

#### Test: `softmax()`

In [ ]:
np.random.seed(0)
test_layer = Layer(0, 'test')
test_layer.net_in = np.random.rand(2, 5)
test_layer.softmax()
print(f'{test_layer.net_act}')

You should get:

`[[0.1955079 0.2308981 0.2063452 0.194741  0.1725077]
 [0.1910613 0.1551337 0.2443186 0.2625293 0.1469572]]`

### 4c. Implement loss function

In `layer.py`, implement cross-entropy loss `cross_entropy()` (see `loss()` for usage). 

Mathematical equation:

$-\frac{1}{B}\sum_{i=1}^B Log \left ( y_{ic} \right )$ 

where $y_{ic}$ is the softmax activation value $y$ for the NEURON CODING THE CORRECT CLASS $c$ for the $i^{th}$ input in the mini-batch ($i: 1...B$).

#### Test: `cross_entropy()`

In [ ]:
np.random.seed(0)
y = np.array([0, 4, 1])
test_layer = Layer(0, 'test')
test_layer.net_in = np.random.rand(3, 5)
test_layer.softmax()
print(f'Your loss is {test_layer.cross_entropy(y)} and it should be 1.7490773084785616')

### 4d. Implement the forward pass of the convolution layer

Do this first because this is the first layer of the network (see above architecture).

Implement and test the following methods in `layer.py`:

- constructor in `Conv2D`
- `compute_net_in` in `Conv2D`
- `forward` in `Layer`. The `forward` method synthesizes your work so far and computes all the forward operations for this (and any other layers you create later on).

##### Test Conv2D initialization

In [ ]:
np.random.seed(0)
conv2_layer = Conv2D(0, 'conv2', n_kers=2, ker_sz=2)
print(f'Your filter weights are\n{conv2_layer.wts}')
print(f'Your bias terms are\n{conv2_layer.b}')

The above should yield:

    Your filter weights are
    [[[[ 0.0176405  0.0040016]
       [ 0.0097874  0.0224089]]

      [[ 0.0186756 -0.0097728]
       [ 0.0095009 -0.0015136]]

      [[-0.0010322  0.004106 ]
       [ 0.0014404  0.0145427]]]


     [[[ 0.0076104  0.0012168]
       [ 0.0044386  0.0033367]]

      [[ 0.0149408 -0.0020516]
       [ 0.0031307 -0.008541 ]]

      [[-0.0255299  0.0065362]
       [ 0.0086444 -0.0074217]]]]
    Your bias terms are
    [ 0.0226975 -0.0145437]

#####  Test `forward` using `Conv2D` layer

In [ ]:
np.random.seed(0)
# Create test net parameters
mini_batch_sz, n_kers, n_chans, ker_sz, img_y, img_x = 1, 2, 3, 4, 5, 5
# Create random test input
inputs = np.random.randn(mini_batch_sz, n_chans, img_y, img_x)

# Create a convolution layer with ReLU activation function
conv_layer = Conv2D(0, 'test', n_kers, ker_sz, n_chans=n_chans, activation='relu')

# Do a forward pass thru the layer
net_act = conv_layer.forward(inputs)

# Extract the computed net values
net_in = conv_layer.net_in
wts = conv_layer.get_wts()
inp = conv_layer.input

print(f'Your input stored in the net has shape: {inp.shape} and it should be (1, 3, 5, 5)')
print(f'Your network wts stored in the net has shape: {wts.shape} and it should be (2, 3, 4, 4)')
print(f'Your network activation has shape: {net_act.shape} and it should be (1, 2, 5, 5)')
print(f'Your net-in has shape: {net_in.shape} and it should be (1, 2, 5, 5)')
print()
print()
print('The first chunk of your filters/weights is:\n', wts[0, 0])
print('The first chunk of your net_in is:\n', net_in[0,0])
print('The first chunk of your net_act is:\n', net_act[0,0])

The expected output above is:

    The first chunk of your filters/weights is:
     [[-0.0068481 -0.008708  -0.0057885 -0.0031155]
     [ 0.0005617 -0.0116515  0.0090083  0.0046566]
     [-0.0153624  0.0148825  0.0189589  0.0117878]
     [-0.0017992 -0.0107075  0.0105445 -0.0040318]]
    The first chunk of your net_in is:
     [[ 0.008935  -0.0421614  0.0154295 -0.0282804 -0.0239351]
     [-0.0038573  0.024427  -0.0244604 -0.0008664 -0.0043392]
     [-0.0366694 -0.1157574 -0.0480979 -0.091434   0.036857 ]
     [-0.0154649 -0.0233789  0.0075369 -0.0789695 -0.0198689]
     [-0.0409501 -0.0217434 -0.0435562  0.1815417 -0.1008181]]
    The first chunk of your net_act is:
     [[0.008935  0.        0.0154295 0.        0.       ]
     [0.        0.024427  0.        0.        0.       ]
     [0.        0.        0.        0.        0.036857 ]
     [0.        0.        0.0075369 0.        0.       ]
     [0.        0.        0.        0.1815417 0.       ]]

### 4e. Implement the forward pass of the max pooling layer

The second layer in the `ConvNet4` architecture is a `MaxPooling2D` layer (uses `MaxPooling2D` to compute `netIn`) that does a max pooling operation on the output (`netAct`) of the previous layer (`Conv2D`).

Implement and test the following methods:
-  `compute_net_in` in `MaxPooling2D`

#####  Test `forward` using `MaxPool2D` layer

In [ ]:
np.random.seed(0)
# Create test net parameters
mini_batch_sz, n_kers, n_chans, ker_sz, img_y, img_x = 1, 2, 3, 4, 6, 6
# Create random test input
inputs = np.random.randn(mini_batch_sz, n_chans, img_y, img_x)

# Create a max pooling layer with default (linear) activation function
pool_layer = MaxPooling2D(0, 'pool', pool_size=2, strides=2)

# Do a forward pass thru the layer
net_act = pool_layer.forward(inputs)

# Extract the computed net values
net_in = pool_layer.net_in
wts = pool_layer.wts
inp = pool_layer.input

print(f'Your input stored in the net has shape: {inp.shape} and it should be (1, 3, 6, 6)')
print(f'Your network wts stored is None (as it should be)? {wts is None}')
print(f'Your network activation has shape: {net_act.shape} and it should be (1, 3, 3, 3)')
print(f'Your net in has shape: {net_in.shape} and it should be (1, 3, 3, 3)')
print()
print()
print('The first chunk of your net_in is:\n', net_in[0,0])
print('The first chunk of your net_act is:\n', net_act[0,0])

The expected output above is:

    The first chunk of your net_in is:
     [[1.7640523 2.2408932 1.867558 ]
     [0.7610377 0.6536186 1.4940791]
     [2.2697546 0.0457585 1.5327792]]
    The first chunk of your net_act is:
     [[1.7640523 2.2408932 1.867558 ]
     [0.7610377 0.6536186 1.4940791]
     [2.2697546 0.0457585 1.5327792]]

### 4f. Implement the forward pass of the Dense layer

The 3rd (hidden) and 4th (output) layers in the `ConvNet4` architecture are ones that use `Dense` `netIn` (these are like the layers in ADALINE/MLP).

Implement and test the following methods:
-  constructor in `Dense`
- `compute_net_in` in `Dense`

##### Test Dense initialization

In [ ]:
np.random.seed(0)
hidden_layer = Dense(0, 'dense', units=10, n_units_prev_layer=3)
print(f'Your filter weights are\n{hidden_layer.wts}')
print(f'Your bias terms are\n{hidden_layer.b}')

The above should yield:

    Your filter weights are
    [[ 0.0017641  0.0004002  0.0009787  0.0022409  0.0018676 -0.0009773
       0.0009501 -0.0001514 -0.0001032  0.0004106]
     [ 0.000144   0.0014543  0.000761   0.0001217  0.0004439  0.0003337
       0.0014941 -0.0002052  0.0003131 -0.0008541]
     [-0.002553   0.0006536  0.0008644 -0.0007422  0.0022698 -0.0014544
       0.0000458 -0.0001872  0.0015328  0.0014694]]
    Your bias terms are
    [ 0.0001549  0.0003782 -0.0008878 -0.0019808 -0.0003479  0.0001563
      0.0012303  0.0012024 -0.0003873 -0.0003023]

#### Test Dense layer forward pass

In [ ]:
np.random.seed(0)
mini_batch_sz, n_kers, n_chans, ker_sz, img_y, img_x = 2, 2, 3, 4, 6, 6
inputs = np.random.randn(mini_batch_sz, n_chans, img_y, img_x)

hidden_layer = Dense(0, 'hidden', units=5, n_units_prev_layer=n_chans*img_y*img_x, activation='relu')
hidden_layer.b -= 0.01

net_act = hidden_layer.forward(inputs)
net_in = hidden_layer.net_in
wts = hidden_layer.wts
inp = hidden_layer.input

print(f'Your input stored in the net has shape: {inp.shape} and it should be (2, 3, 6, 6)')
print(f'Your network wts have shape {wts.shape} and it should be (108, 5)')
print(f'Your network activation has shape: {net_act.shape} and it should be (2, 5)')
print(f'Your net in has shape: {net_in.shape} and it should be (2, 5)')
print()
print()
print('Your net_in is:\n', net_in)
print('Your net_act is:\n', net_act)

The expected output above is:

    Your net_in is:
     [[-0.028588   0.0268116  0.0015849 -0.0014458  0.0083166]
     [-0.0130934  0.0027427 -0.0164959 -0.0100963 -0.0365421]]
    Your net_act is:
     [[0.        0.0268116 0.0015849 0.        0.0083166]
     [0.        0.0027427 0.        0.        0.       ]]

### 4g. Implement network full forward pass

Now it's time to chain all the individual layers together into a network. 

In `network.py`, implement and the following methods:

- `forward`. This is the forward pass in the that calls each layer's `forward` method that you implemented above. The result of this method will be the loss derived from the activation (`net_act`) of the Output layer, 4 layers deep.
- `wt_reg_reduce`. This is needed for a complete implementation of the full forward method.

Before you can test the forward pass of the network, you need to define what layers belong in the network and how they are arranged! This is done by making a subclass of `Network`. 

- Implement the constructor of `ConvNet4`, adding the layers (in forward pass order):

Conv2D $->$ MaxPooling2D $->$ Dense $->$ Dense

##### Test network forward pass

In [ ]:
from network import ConvNet4

In [ ]:
np.random.seed(0)
n_inputs = 5
X = np.random.randn(n_inputs, 3, 32, 32)
y = np.random.randint(10, size=n_inputs)

net = ConvNet4()
loss = net.forward(X, y)
print(f'Forward testing loss is {loss} and it should be 2.3027537235484816')

net.reg = 1.0
loss = net.forward(X, y)
print(f'Forward testing regularized loss is {loss} and it should be 2.714961739709542')
print()
print(f'Your output layer activation values are\n{net.layers[-1].net_act}')

The above should print:

    Your output layer activation values are
    [[0.1000324 0.100044  0.0999474 0.0999674 0.0999955 0.1000346 0.1000667
      0.1000034 0.0999716 0.0999369]
     [0.1000312 0.1000442 0.0999474 0.099968  0.0999964 0.1000353 0.1000669
      0.1000023 0.0999724 0.0999359]
     [0.1000313 0.1000452 0.0999472 0.0999672 0.0999964 0.100035  0.1000662
      0.1000027 0.0999722 0.0999365]
     [0.1000321 0.1000442 0.099947  0.0999672 0.0999966 0.1000352 0.1000667
      0.1000022 0.0999723 0.0999366]
     [0.1000318 0.1000454 0.0999471 0.0999673 0.0999959 0.1000343 0.100066
      0.1000025 0.0999728 0.099937 ]]

### 4h. Implement the backward pass of the convolutional neural network

Next, you are going to implement the backward pass of gradients that stem from the loss function and propogate all the way to the 1st layer of the network. 

As usual, we need to compute several types of gradients for EACH network layer (see instance variable placeholders in the constructor):
- `d_net_act`
- `d_net_wts` (for layers that have weights)
- `d_net_in`
- `d_b` (for layers that have weights)

#### The flow of the backward gradients

- `d_upstream` gives us the gradient from the layer above that GOT US TO the `net_act` stage of the new, current layer. Using `d_upstream`, we compute `d_net_in` via (`backward_netAct_to_netIn` in `Layer`) — this gets us to the `net_in` stage, like usual.
- Using `d_net_in`, we compute [`dprev_net_act`, `d_net_wts`, `d_b`] via `backward_netIn_to_prevLayer_netAct` in `Layer`, where `dprev_net_act` is the net_act gradient for layer beneath the current one (the `d_upstream` for the one layer down).
- `dprev_net_act` becomes `d_upstream` for the next layer down, and the process repeats...

We only need to "store" `d_b` and `d_net_wts` as instance variables in a layer because these are needed for weight updates during training/backprop, the other variables are just needed temporarily as a means to compute `d_b` and `d_net_wts` in areas downstream. 

#### Goal of backward pass

We need to compute these variables (`d_net_act` `d_net_wts` `d_net_in` `d_b`) for each network layer. Start working at the network level and drill down into the layer-specific implementations.

Implement the following:

- `backward` in `Network`
- `backward` in `Layer`
- `backward_netAct_to_netIn` in `Layer` (computes `d_net_in`)
- `backward_netIn_to_prevLayer_netAct` in `Dense` (computes [`dprev_net_act`, `d_net_wts`, `d_b`])
- `backward_netIn_to_prevLayer_netAct` in `MaxPooling2D` (arguably the most challenging, so there are more detailed instructions)

(`backward_netIn_to_prevLayer_netAct` in `Conv2D` is already done for you :)

#### 4h. (i) Test backwards thru output (Dense) layer

In [ ]:
np.random.seed(0)
n_inputs = 5
n_hidden = 10
n_chans, img_y, img_x = 1, 3, 3
n_units_prev_layer = n_chans*img_y*img_x

# Define test inputs/net quantities
inputs = np.random.rand(n_inputs, n_chans, img_y, img_x)  # 5, 1, 3, 3
wts = np.random.rand(n_units_prev_layer, n_hidden)  # 9, 10
b = np.random.rand(n_hidden)  # 10
d_upstream = np.random.rand(n_inputs, n_hidden)  # 5, 10

# Create layer and fill it with the test values
dense_layer = Dense(3, 'Output', units=n_hidden, n_units_prev_layer=n_units_prev_layer)
dense_layer.input = inputs
dense_layer.wts = wts
dense_layer.b = b
dense_layer.verbose = False  # Toggle this on/off as needed
dense_layer.compute_net_in()
dense_layer.compute_net_act()

# Do the backwards pass thru the layer
dprev_net_act, d_wts, d_b = dense_layer.backward_netIn_to_prevLayer_netAct(d_upstream)
print(f'Shapes: d_b {d_b.shape}, d_wts {d_wts.shape}, and dprev_net_act {dprev_net_act.shape}')
print(f'Shapes should be: d_b (10,), d_wts (9, 10), and (5, 1, 3, 3)')

print()
print(f'Your d_b is\n{d_b}')
print()
print(f'Your d_wts is\n{d_wts}')
print()
print(f'Your dprev_net_act is\n{dprev_net_act}')
print()

**The above gradients should be:**

    Your d_b is
    [3.2455908 1.8002444 3.3665527 3.2859516 1.5834276 1.9308289 2.2073289
     1.8479667 2.6827289 3.566804 ]

    Your d_wts is
    [[1.9647549 1.0277824 2.2407716 2.2285063 0.9292745 1.1542214 1.4081657
      1.1809309 1.4872235 2.3948296]
     [2.4149935 1.2317185 2.269614  2.2863302 1.1275105 1.4439853 1.4810075
      1.2779767 1.8490518 2.4951982]
     [2.3825726 1.1475702 2.1490725 2.3414029 1.1142103 1.4129572 1.2720702
      1.0906306 1.8372825 2.4526423]
     [2.0270211 0.9791365 1.7597065 1.8880233 0.9418178 1.2090793 1.0815757
      0.9381876 1.5491818 2.0072897]
     [1.8721506 1.2550663 2.0130525 1.8242208 0.7031881 1.2199752 1.3692692
      1.3433151 1.5949467 2.3108458]
     [1.5533222 0.5228488 1.5964426 1.7004334 0.934878  0.8019153 1.0105438
      0.6570236 0.9948233 1.5181254]
     [1.0387269 0.6855684 1.4336079 1.3441286 0.6991294 0.5592176 0.9254875
      0.6156819 1.1410889 1.2800253]
     [1.2336846 0.1519462 1.1147997 1.1695178 1.021358  0.5078748 0.8424287
      0.3095472 0.639654  0.7001401]
     [2.0547378 1.3408975 2.3780681 2.0538556 1.2724045 1.1666446 1.7457629
      1.2671205 2.0938212 2.11545  ]]

    Your dprev_net_act is
    [[[[1.9155271 1.3247292 2.3009418]
       [1.4105462 2.7904128 1.8812208]
       [3.301176  3.371235  2.2080999]]]


     [[[2.5679488 1.961313  4.2226248]
       [2.0673985 2.5227133 3.1804764]
       [3.7013106 4.1802547 2.5279694]]]


     [[[1.601765  0.9979089 2.3158531]
       [0.9802312 1.318577  1.5069402]
       [2.102651  2.2660942 1.9897509]]]


     [[[1.7326733 1.5231588 2.8999431]
       [1.4712951 1.7467105 2.0114005]
       [2.8864938 3.1077611 1.9464542]]]


     [[[1.6982242 1.4995445 3.1521748]
       [1.7981848 2.3909886 2.6012807]
       [2.8371598 3.4117499 2.1974959]]]]



#### 4h. (ii) Test backwards thru output (MaxPool2D) layer

In [ ]:
np.random.seed(0)
n_inputs = 3

# Define test inputs/net quantities
inputs = np.random.rand(n_inputs, 3, 4, 4)
d_upstream = np.random.rand(n_inputs, 3, 2, 2)

pool_sz = 2
stride = 2

# Create layer and fill it with the test values
pool_layer = MaxPooling2D(1, 'Pool', pool_size=pool_sz, strides=stride)

# Do the forward/backwards pass thru the layer
pool_layer.verbose = False
pool_layer.forward(inputs)
dprev_net_act, _, _ = pool_layer.backward(d_upstream, None)

print(f'Shape: {dprev_net_act.shape}')
print(f'Shape should be: (3, 3, 4, 4)')

print()
print(f'Your d_net_in is\n{dprev_net_act}')

**The above gradients should be:**
    
    Your d_net_in is
    [[[[0.        0.806194  0.        0.       ]
       [0.        0.        0.        0.7038886]
       [0.1002269 0.        0.9194826 0.       ]
       [0.        0.        0.        0.       ]]

      [[0.        0.        0.        0.998847 ]
       [0.7142413 0.        0.        0.       ]
       [0.        0.1494483 0.        0.8681261]
       [0.        0.        0.        0.       ]]

      [[0.        0.        0.        0.       ]
       [0.        0.1624929 0.6155596 0.       ]
       [0.        0.        0.8480082 0.       ]
       [0.        0.12382   0.        0.       ]]]


     [[[0.        0.        0.5691007 0.       ]
       [0.807319  0.        0.        0.       ]
       [0.4071833 0.        0.        0.       ]
       [0.        0.        0.069167  0.       ]]

      [[0.        0.        0.        0.       ]
       [0.6974288 0.        0.4535427 0.       ]
       [0.7220556 0.        0.8663823 0.       ]
       [0.        0.        0.        0.       ]]

      [[0.        0.        0.        0.8558033]
       [0.9755215 0.        0.        0.       ]
       [0.        0.0117141 0.        0.3599781]
       [0.        0.        0.        0.       ]]]


     [[[0.        0.        0.        0.       ]
       [0.7299906 0.        0.        0.1716297]
       [0.        0.        0.        0.       ]
       [0.        0.5210366 0.        0.054338 ]]

      [[0.        0.        0.        0.       ]
       [0.1999965 0.        0.0185218 0.       ]
       [0.7936977 0.        0.2239247 0.       ]
       [0.        0.        0.        0.       ]]

      [[0.3453517 0.        0.9280813 0.       ]
       [0.        0.        0.        0.       ]
       [0.        0.        0.        0.       ]
       [0.7044144 0.        0.        0.0318389]]]]

#### 4h. (iii) Test network full backwards pass

In [ ]:
np.random.seed(0)
n_inputs = 2
X = np.random.randn(n_inputs, 3, 32, 32)
y = np.random.randint(10, size=n_inputs)

# Do forwards and backwards pass thru network
net = ConvNet4()
loss = net.forward(X, y)
net.backward(y)

# Check various gradients in each layer
print('Output layer')
print('------------------------------------')
print(f'd_wts (1st chunk):\n{net.layers[-1].d_wts[0]}\n')
print(f'd_b (all):\n{net.layers[-1].d_b}\n')
print('------------------------------------')
print('Dense hidden layer')
print('------------------------------------')
print(f'd_wts (1st chunk):\n{net.layers[-2].d_wts[0]}\n')
print(f'd_b (all):\n{net.layers[-2].d_b}\n')
print('------------------------------------')
print('Con2D layer')
print('------------------------------------')
print(f'd_wts (1st chunk):\n{net.layers[0].d_wts[0,0]}\n')
print(f'd_b (all):\n{net.layers[0].d_b}\n')
print('------------------------------------')

**Above output should be:**

    Output layer
    ------------------------------------
    d_wts (1st chunk):
    [-0.0011711  0.000251   0.0002509 -0.000838   0.0002515  0.0002509
      0.0002513  0.0002513  0.0002508  0.0002514]

    d_b (all):
    [-0.4000438  0.0999714  0.0999342 -0.4000436  0.1001569  0.0999098
      0.1000612  0.1000623  0.0998681  0.1001235]

    ------------------------------------
    Dense hidden layer
    ------------------------------------
    d_wts (1st chunk):
    [ 0.0000032  0.0000012 -0.000003  -0.0000017  0.         0.
     -0.0000011  0.0000033  0.         0.0000035  0.         0.
      0.         0.         0.0000008  0.        -0.0000007  0.0000009
      0.0000018 -0.0000012  0.         0.        -0.0000005 -0.0000035
      0.0000056  0.0000002  0.0000004  0.0000007  0.000002   0.
     -0.0000024  0.0000047  0.0000015 -0.0000002 -0.0000052 -0.0000006
     -0.0000017  0.         0.        -0.0000011  0.         0.
      0.0000026  0.0000011 -0.0000033 -0.0000007  0.0000018 -0.000002
      0.        -0.0000003  0.0000046  0.         0.         0.0000016
      0.         0.0000069  0.0000008  0.         0.         0.0000002
      0.         0.0000004 -0.0000055 -0.0000021 -0.0000008  0.0000027
      0.0000017  0.         0.         0.         0.         0.0000019
      0.        -0.0000027  0.         0.        -0.0000037  0.
      0.         0.         0.         0.0000015  0.0000036  0.0000011
      0.         0.0000009 -0.0000015  0.        -0.0000006  0.
      0.0000007  0.         0.         0.         0.        -0.000003
     -0.0000018  0.        -0.0000017  0.       ]

    d_b (all):
    [ 0.0008354  0.0002864 -0.0008092 -0.0003523  0.         0.
     -0.0003092  0.0007596  0.         0.0008171  0.         0.
      0.         0.         0.0002196  0.        -0.000181   0.0002
      0.0004072 -0.0003272  0.         0.        -0.0001368 -0.0008862
      0.0013209  0.0000489  0.000079   0.0002014  0.0004692  0.
     -0.0006495  0.001113   0.0003588  0.0000003 -0.0012009 -0.000128
     -0.0004578  0.         0.        -0.0002509  0.         0.
      0.0007343  0.0002856 -0.0007659 -0.0002002  0.000457  -0.000389
      0.        -0.0000904  0.0010889  0.         0.         0.000357
      0.         0.0017132  0.0002065  0.         0.         0.0000985
      0.         0.0000826 -0.0013527 -0.0006262 -0.0002362  0.0007406
      0.0004243  0.         0.         0.         0.         0.0003552
      0.        -0.0007183  0.         0.        -0.0008621  0.
      0.         0.         0.         0.0004168  0.000966   0.0002825
      0.         0.0001653 -0.000392   0.0000081 -0.0000571  0.
      0.0001575  0.         0.         0.         0.        -0.0006998
     -0.0004846  0.        -0.000399   0.       ]

    ------------------------------------
    Con2D layer
    ------------------------------------
    d_wts (1st chunk):
    [[ 0.0000488  0.0000134  0.0000127 -0.000047  -0.0000053  0.0000121
      -0.0000369]
     [-0.0000679  0.0000001  0.0000846 -0.0000738 -0.0000172 -0.0000509
      -0.0000184]
     [ 0.0000623  0.0001172  0.000046  -0.0000286  0.0001153 -0.0000021
      -0.0001069]
     [ 0.0000919 -0.0000284  0.0000972  0.0000489 -0.0001472 -0.0000002
      -0.0001246]
     [-0.0000035 -0.000133   0.0000469  0.000036  -0.000083  -0.0001644
       0.0000074]
     [ 0.0000059  0.0000673 -0.00004   -0.0000633  0.0000487  0.0001111
       0.0000047]
     [ 0.0000116  0.0001     0.0001376 -0.0000285  0.0000712  0.0000673
       0.0000697]]

    d_b (all):
    [ 0.0000062 -0.0000343  0.0000214  0.0000532 -0.0000348  0.0000877
      0.0000694 -0.0000575  0.0000409 -0.0000378  0.0001747  0.0000727
      0.0000187  0.0000425  0.0001503 -0.0001182  0.000006  -0.000016
     -0.0000527  0.0000294 -0.0001027  0.0000381 -0.000024  -0.0000202
      0.0000533  0.0000042 -0.0000012 -0.0000294 -0.0000433 -0.0000949
      0.0001349 -0.0000746]

    ------------------------------------